# Biostat 257 Homework 3

**Due May 14 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.6.0)
  CPU: Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, westmere)


## Q1. Big $n$ linear regression

People often think linear regression on a dataset with millions of observations is a big data problem. Now we learnt various methods for solving linear regression and should realize that, with right choice of algorithm, it is a problem that can be handled by any moderate computer.

### Q1.1 Download data (10pts)

Download the flight data from <https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7>. **Do not put data files in Git.** You will lose points if you do. For grading purpose (reproducibility), we will assume that data files are in a subfolder `flights`.

## Sol Q1.1

In [2]:
using CodecBzip2, CSV, DataFrames, Distributions, LinearAlgebra, 
Serialization, StatsModels, SweepOperator
ENV["COLUMNS"] = 200

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1317
┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1317
┌ Info: Precompiling StatsModels [3eaba693-59b7-5ba5-a881-562e759f1c8d]
└ @ Base loading.jl:1317


200

In [3]:
;ls -l flights

total 3095696
-rw-rw-r--@ 1 elviscui  staff   12652442 May  3 03:08 1987.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   49499025 May  3 03:08 1988.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   49202298 May  3 03:09 1989.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   52041322 May  3 03:09 1990.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   49877448 May  3 03:10 1991.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   50040946 May  3 03:10 1992.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   50111774 May  3 03:10 1993.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   51123887 May  3 03:11 1994.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   74881752 May  3 03:11 1995.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   75887707 May  3 03:11 1996.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   76705687 May  3 03:12 1997.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   76683506 May  3 03:12 1998.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   79449438 May  3 03:12 1999.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   82537924 May  3 03:12 2000.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   83478700 May  3 

Find out how many data points are in each year.

**I personally would suggest check the "effective" observations in the model fitting step, since we are going to drop a lot of obs.**

In [4]:
files = readdir("flights/")[1:22]

for file in files
    open(string("flights/", file), "r") do io
        num = countlines(Bzip2DecompressorStream(io))
        print(file, ", ", num, "\n")
    end
end

1987.csv.bz2, 1311827
1988.csv.bz2, 5202097
1989.csv.bz2, 5041201
1990.csv.bz2, 5270894
1991.csv.bz2, 5076926
1992.csv.bz2, 5092158
1993.csv.bz2, 5070502
1994.csv.bz2, 5180049
1995.csv.bz2, 5327436
1996.csv.bz2, 5351984
1997.csv.bz2, 5411844
1998.csv.bz2, 5384722
1999.csv.bz2, 5527885
2000.csv.bz2, 5683048
2001.csv.bz2, 5967781
2002.csv.bz2, 5271360
2003.csv.bz2, 6488541
2004.csv.bz2, 7129271
2005.csv.bz2, 7140597
2006.csv.bz2, 7141923
2007.csv.bz2, 7453216
2008.csv.bz2, 2389218


### Q1.2 (10 pts) Problem size

We are interested in how the arrival delay of a flight, `ArrDelay`, depends on the distance traveled (`Distance`), departure delay (`DepDelay`), weekday (`DayOfWeek`), and airline (`UniqueCarrier`). 

We want to fit a linear regression `ArrDelay ~ 1 + Distance + DepDelay + DayOfWeek + UniqueCarrier` using data from 1987-2008. Treat `DayOfWeek` as a factor with 6 levels. We use the dummy coding with `1` (Monday) as the base level. Treat `UniqueCarrier` as a factor with 8 levels: "AA", "AS", "CO", "DL", "NW", "UA", "US", and "WN". We use the dummy coding with "AA" as the base level.

Will the design matrix $\mathbf{X}$ (in double precision) fit into the memory of your computer? 

## Sol Q1.2

- My computer memory: 16 GB [MacBook Pro (13-inch, M1, 2020)].
- Next I calculate the size of all files in **flights**.

In [5]:
total_size = 0

for file in files
    total_size += filesize("flights/" * file)
end

print("The total size of all files is: ", round(total_size / (1_000_000_000), digits=2), " GB.")

The total size of X is: 1.58 GB.

- **Conclusion:** 
Since DoublePrecision costs 64 bits, or 8 Bytes, so the total storage required should be around

    $$1.58\times 8\approx 12.8GB<16GB$$
    
$$\text{It seems that the design matrix $X$ CAN fit into the memory of my computer.}$$

### Q1.3 (30 pts) Choose algorithm

Assume your computer has limited memory, say only 1GB. Review the [Summary of Linear Regression](https://ucla-biostat-257-2021spring.github.io/slides/15-linreg/linreg.html) and choose one method in the table to solve the linear regression.

Report the estimated regression coefficients $\widehat \beta$, estimated variance $\widehat \sigma^2 = \sum_i (y_i - \widehat y_i)^2 / (n - p)$, and standard errors of $\widehat \beta$.

Hint: It took my laptop about 10-11 minutes to import data and fit linear regression.

## Sol Q1.3

In [6]:
# preparation

airlines = ["AA", "AS", "CO", "DL", "NW", "UA", "US", "WN"]

Xty = zeros(16)
XtX = zeros(16, 16)

obs_count = 0

0

In [12]:
for year in 1987:2008
    
    print("Year " * string(year) * " costs:")
    
    # load data from different years into DataFrame
    @time df = open("flights/" * string(year) * ".csv.bz2", "r") do io
        CSV.File(
            Bzip2DecompressorStream(io), 
            select = ["DayOfWeek", "UniqueCarrier", "ArrDelay", 
                "DepDelay", "Distance"],
            types = Dict(
                "DayOfWeek" => UInt8,
                "UniqueCarrier" => String, 
                "ArrDelay" => Float64, 
                "DepDelay" => Float64, 
                "Distance" => UInt16
                ),
            missingstring = "NA"
            ) |> DataFrame
    end 

    filter!(row -> row[:UniqueCarrier] ∈ airlines, df)
    dropmissing!(df)

    # model frame for year 2003
    mf = ModelFrame(
        @formula(ArrDelay ~ 1 + DayOfWeek + Distance + DepDelay + UniqueCarrier), 
        df,
        contrasts = Dict(
            :DayOfWeek => StatsModels.DummyCoding(base = 1, levels = UInt8.(1:7)),
            :UniqueCarrier => StatsModels.DummyCoding(
                base = "AA", 
                levels = ["AA", "AS", "CO", "DL", "NW", "UA", "US", "WN"]
            )
        )
    )

    # generate the covariate matrix X for year 2003
    X = modelmatrix(mf)

    # generate the response vector Y for year 2003
    y = df[!, :ArrDelay]

    Xty += X'y
    XtX += X'X
    obs_count += length(y)
    
end

print("Total count of obs is: ", obs_count)

Year 1987 costs:  2.836882 seconds (1.33 M allocations: 253.548 MiB, 0.80% compilation time)
Year 1988 costs: 11.059098 seconds (5.20 M allocations: 989.537 MiB, 0.93% gc time)
Year 1989 costs: 10.707616 seconds (5.04 M allocations: 968.166 MiB, 0.54% gc time)
Year 1990 costs: 11.507792 seconds (5.27 M allocations: 998.383 MiB, 1.94% gc time)
Year 1991 costs: 11.039466 seconds (5.08 M allocations: 975.487 MiB, 2.30% gc time)
Year 1992 costs: 11.336809 seconds (5.09 M allocations: 988.190 MiB, 2.86% gc time)
Year 1993 costs: 11.170045 seconds (5.07 M allocations: 979.221 MiB, 3.90% gc time)
Year 1994 costs: 11.539568 seconds (5.18 M allocations: 991.988 MiB, 4.21% gc time)
Year 1995 costs: 12.949489 seconds (5.33 M allocations: 749.569 MiB, 2.11% gc time)
Year 1996 costs: 13.308444 seconds (5.35 M allocations: 1002.870 MiB, 3.23% gc time)
Year 1997 costs: 13.474848 seconds (5.41 M allocations: 1.495 GiB, 1.96% gc time)
Year 1998 costs: 13.504214 seconds (5.39 M allocations: 764.628 MiB,

## LU decomposition solution

- $2p^3+2p^2-p$ or $O(p^3)$ flops.

In [13]:
inv(XtX) * Xty

16-element Vector{Float64}:
  0.8356103659418359
  0.25913928443937373
  0.6686823912995331
  1.0157317379437227
  0.8120924952457038
 -1.611798446799804
 -0.7046792815400416
 -0.0014099093758319333
  0.9249347505487564
  1.039884764440199
  0.25793989036200315
  0.6621510212696293
  0.034091184632845994
  0.04029170087803635
 -0.4523526244199978
 -2.9759959529106013

## Cholesky decomposition solution
- Approximately $\frac{p^3}{3}+2p^2$ flops, or $O(p^3)$ flops.

In [15]:
cholesky(XtX) \ Xty

16-element Vector{Float64}:
  0.8356103659428438
  0.25913928443890344
  0.6686823912992311
  1.0157317379431565
  0.8120924952456089
 -1.6117984467998752
 -0.7046792815413438
 -0.0014099093758318739
  0.9249347505487558
  1.0398847644401354
  0.257939890361988
  0.6621510212696275
  0.034091184632955684
  0.04029170087785372
 -0.45235262442005325
 -2.9759959529105626

## Sweep operator solution
- This one is easy to calculate:
    nothing but $p^3$ flops.

In [16]:
using SweepOperator

hi = [XtX Xty; Xty' 1]
sweep!(hi, 1:16)[1:16, end]

16-element Vector{Float64}:
  0.8356103659428409
  0.2591392844389043
  0.6686823912992305
  1.015731737943157
  0.812092495245611
 -1.6117984467998785
 -0.7046792815413438
 -0.0014099093758318726
  0.9249347505487558
  1.0398847644401357
  0.25793989036198806
  0.6621510212696293
  0.03409118463295879
  0.04029170087785405
 -0.452352624420052
 -2.975995952910561

### Q1.4 Be proud of yourself

Go to your resume/cv and claim you have experience performing analytics on data with 100 million observations.

## Sol Q1.4

- Haven't done yet.
- But it counts 0 point.
- So I will add it to my CV later.
- Indeed this is an excellent project.

### Sample code

Following code explores the data in 2003 and generates the design matrix and responses for that year. Feel free to use the code in your solution.

In [3]:
#using CodecBzip2, CSV, DataFrames, Distributions, LinearAlgebra, 
#Serialization, StatsModels, SweepOperator
#ENV["COLUMNS"] = 200

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1317
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1317
┌ Info: Precompiling StatsModels [3eaba693-59b7-5ba5-a881-562e759f1c8d]
└ @ Base loading.jl:1317


200

In [4]:
# Print first 10 lines of 2003 data.
run(pipeline(`bunzip2 -c flights/2003.csv.bz2`, `head`))

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2003,1,29,3,1651,1655,1912,1913,UA,1017,N202UA,141,138,119,-1,-4,ORD,MSY,837,5,17,0,NA,0,NA,NA,NA,NA,NA
2003,1,30,4,1654,1655,1910,1913,UA,1017,N311UA,136,138,108,-3,-1,ORD,MSY,837,2,26,0,NA,0,NA,NA,NA,NA,NA
2003,1,31,5,1724,1655,1936,1913,UA,1017,N317UA,132,138,110,23,29,ORD,MSY,837,5,17,0,NA,0,NA,NA,NA,NA,NA
2003,1,1,3,1033,1035,1625,1634,UA,1018,N409UA,232,239,215,-9,-2,OAK,ORD,1835,6,11,0,NA,0,NA,NA,NA,NA,NA
2003,1,2,4,1053,1035,1726,1634,UA,1018,N496UA,273,239,214,52,18,OAK,ORD,1835,13,46,0,NA,0,NA,NA,NA,NA,NA
2003,1,3,5,1031,1035,1640,1634,UA,1018,N412UA,249,239,223,6,-4,OAK,ORD,1835,13,13,0,NA,0,NA,NA,NA,NA,NA
2003,1,4,6,1031,1035,1626,1634,UA,1018,N455UA,235,239,219,-8,-4,OAK,ORD,183

Base.ProcessChain(Base.Process[Process(`bunzip2 -c flights/2003.csv.bz2`, ProcessSignaled(13)), Process(`head`, ProcessExited(0))], Base.DevNull(), Base.DevNull(), Base.DevNull())

In [5]:
# how many data points in 2003?
open("flights/2003.csv.bz2", "r") do io
    countlines(Bzip2DecompressorStream(io))
end

6488541

In [6]:
# # figure out which airlines appear in each year of 1987-2008
# airlines = Vector{Vector{String}}(undef, 22)
# @time for year in 1987:2008
#     println("year $year")
#     filename = "flights/" * string(year) * ".csv.bz2"
#     df = open(filename, "r") do io
#         CSV.File(
#             Bzip2DecompressorStream(io),
#             select = ["UniqueCarrier"],
#             types = Dict("UniqueCarrier" => String),
#             missingstring = "NA"
#         ) |> DataFrame
#     end
#     airlines[year - 1986] = unique(df[!, :UniqueCarrier])
# end
# intersect(airlines...) |> sort

In [7]:
# load 2003 data into DataFrame
@time df = open("flights/2003.csv.bz2", "r") do io
    CSV.File(
        Bzip2DecompressorStream(io), 
        select = ["DayOfWeek", "UniqueCarrier", "ArrDelay", 
            "DepDelay", "Distance"],
        types = Dict(
            "DayOfWeek" => UInt8,
            "UniqueCarrier" => String, 
            "ArrDelay" => Float64, 
            "DepDelay" => Float64, 
            "Distance" => UInt16
            ),
        missingstring = "NA"
        ) |> DataFrame
end

 32.675195 seconds (39.79 M allocations: 4.854 GiB, 2.26% gc time, 24.62% compilation time)


,DayOfWeek,UniqueCarrier,ArrDelay,DepDelay,Distance
,UInt8,String,Float64?,Float64?,UInt16
1,3,UA,-1.0,-4.0,837
2,4,UA,-3.0,-1.0,837
3,5,UA,23.0,29.0,837
4,3,UA,-9.0,-2.0,1835
5,4,UA,52.0,18.0,1835
6,5,UA,6.0,-4.0,1835
7,6,UA,-8.0,-4.0,1835
8,7,UA,2.0,0.0,1835
9,1,UA,19.0,-4.0,1835


In [8]:
# how many rows?
size(df, 1)

6488540

In [9]:
# drop rows with missing values
dropmissing!(df)
size(df, 1)

6375689

In [10]:
# filter out rows not in the airline list
airlines = ["AA", "AS", "CO", "DL", "NW", "UA", "US", "WN"]
filter!(row -> row[:UniqueCarrier] ∈ airlines, df)
size(df, 1)

4230285

In [ ]:
# model frame for year 2003
mf = ModelFrame(
    @formula(ArrDelay ~ 1 + DayOfWeek + Distance + DepDelay + UniqueCarrier), 
    df,
    contrasts = Dict(
        :DayOfWeek => StatsModels.DummyCoding(base = 1, levels = UInt8.(1:7)),
        :UniqueCarrier => StatsModels.DummyCoding(
            base = "AA", 
            levels = ["AA", "AS", "CO", "DL", "NW", "UA", "US", "WN"]
        )
    )
)

In [12]:
# generate the covariate matrix X for year 2003
X = modelmatrix(mf)

4230285×16 Matrix{Float64}:
 1.0  0.0  1.0  0.0  0.0  0.0  0.0   837.0   -4.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0  0.0   837.0   -1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0  0.0   837.0   29.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0  0.0  1835.0   -2.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0  0.0  1835.0   18.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0  0.0  1835.0   -4.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  1835.0   -4.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  1835.0    0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  1835.0   -4.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0  0.0   413.0    3.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0  0.0   413.0   -4.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0  0.0   

In [13]:
# generate the response vector Y for year 2003
y = df[!, :ArrDelay]

4230285-element Vector{Float64}:
  -1.0
  -3.0
  23.0
  -9.0
  52.0
   6.0
  -8.0
   2.0
  19.0
   4.0
 -23.0
 -19.0
 -12.0
   ⋮
 108.0
  66.0
  62.0
  66.0
  27.0
 134.0
  53.0
  47.0
  54.0
  -5.0
   3.0
  -1.0

## Q2. PageRank

We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

### Q2.1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

### Q2.2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

### Q2.3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [17]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

┌ Info: Precompiling MatrixDepot [b51810bb-c9f3-55da-ae3c-350fc1fbce05]
└ @ Base loading.jl:1317


include group.jl for user defined matrix generators
verify download of index files...
reading database
adding metadata...
adding svd data...
writing database
used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [18]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:
* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  
* number of web pages
* number of edges (web links). 
* number of dangling nodes (pages with no out links)
* histogram of in-degrees  
* list the top 20 pages with the largest in-degrees?  
* histogram of out-degrees
* which the top 20 pages with the largest out-degrees?
* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

### Q2.4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

### Q2.5 Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [16]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    # TODO: initialize and pre-allocate arrays
    PageRankImPt(A, telep)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
Base.getindex(M::PageRankImPt, i, j) = M.telep

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}) where T <: AbstractFloat
    # TODO: implement mul!(out, M, v)
    sleep(1e-2) # wait 10 ms as if your code takes 1ms
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    # TODO: implement mul!(out, transpose(M), v)
    sleep(1e-2) # wait 10 ms as if your code takes 1ms
    out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [17]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257-2021spring/ucla-biostat-257-2021spring.github.io/master/hw/hw3/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [18]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

@assert transpose(M) * ones(n) ≈ zeros(n)

In [19]:
@assert M * xsol ≈ zeros(n)

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate little memory.

In [20]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  144 bytes
  allocs estimate:  5
  --------------
  minimum time:     10.274 ms (0.00% GC)
  median time:      12.783 ms (0.00% GC)
  mean time:        12.790 ms (0.00% GC)
  maximum time:     14.319 ms (0.00% GC)
  --------------
  samples:          355
  evals/sample:     1

In [21]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  144 bytes
  allocs estimate:  5
  --------------
  minimum time:     10.146 ms (0.00% GC)
  median time:      12.720 ms (0.00% GC)
  mean time:        12.644 ms (0.00% GC)
  maximum time:     14.118 ms (0.00% GC)
  --------------
  samples:          359
  evals/sample:     1

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [22]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

17.12

**Hint**: My median run times are 30-40 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [23]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.5373439728225696e-5, 1.1625074089088258e-6, 7.4732619144138796e-6, 6.642899479479004e-7, 9.964349219218506e-7, 2.6571597917916015e-6, 1.660724869869751e-7, 6.642899479479004e-7, 3.321449739739502e-7, 3.321449739739502e-7  …  2.989304765765552e-6, 1.3285798958958008e-6, 2.4910873048046265e-6, 4.982174609609253e-7, 1.660724869869751e-7, 2.4910873048046265e-6, 3.321449739739502e-7, 1.660724869869751e-7, 7.4732619144138796e-6, 4.982174609609253e-7], ConvergenceInfo: one converged value after 0 iterations and 1 applications of the linear map;
norms of residuals are given by (0.0,).
), time = 0.068877865, bytes = 27636729, gctime = 0.017346028, gcstats = Base.GC_Diff(27636729, 3, 0, 79377, 3, 0, 17346028, 1, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [24]:
@assert norm(x_gmres - xsol) < 1e-8

LoadError: AssertionError: norm(x_gmres - xsol) < 1.0e-8

GMRES should be reasonably fast. The points you'll get is

In [25]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 7-8 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [26]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = ([0.0], [[0.0057161240427034184, 0.0001878538417789856, 0.0012076318400077645, 0.00010734505244513462, 0.00016101757866770192, 0.00042938020978053847, 2.6836263111283654e-5, 0.00010734505244513462, 5.367252622256731e-5, 5.367252622256731e-5  …  0.00048305273600310583, 0.00021469010489026923, 0.00040254394666925487, 8.050878933385096e-5, 2.6836263111283654e-5, 0.00040254394666925487, 5.367252622256731e-5, 2.6836263111283654e-5, 0.0012076318400077645, 8.050878933385096e-5]], ConvergenceInfo: one converged value after 1 iterations and 1 applications of the linear map;
norms of residuals are given by (0.0,).
), time = 0.089409929, bytes = 34596553, gctime = 0.0152803, gcstats = Base.GC_Diff(34596553, 4, 0, 79217, 16, 0, 15280300, 1, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [27]:
@assert abs(Real(vals[1])) < 1e-8

In [28]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

LoadError: AssertionError: norm(x_arnoldi - xsol) < 1.0e-8

Arnoldi should be reasonably fast. The points you'll get is

In [29]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 11-12 seconds.

### Q2.6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

### Q2.7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.